In [2]:
import numpy as np
import pandas as pd

path='C:/Users/atousaz/Box/Sharma/1week/Dodge_street/data/'

date_list=['3','4','5','6','7']

In [4]:

for i in date_list:
    df=pd.read_csv(path+'concatenated_df_'+i+'.csv')



    df['datetime']=pd.to_datetime(df['datetime'])
    cycle_df=df[(df['event']==150) & (df['parameter']==5)]
    cycle_df=cycle_df.sort_values(['signal','datetime']) 
    cycle_df['datetime_shift']=cycle_df.groupby(['signal'])['datetime'].shift(-1)
    cycle_df=cycle_df.rename(columns={'datetime':'start_cycle','datetime_shift':'end_cycle'})
    cycle_df['cycle_lenght']=(cycle_df['end_cycle']-cycle_df['start_cycle']).dt.total_seconds()
    cycle_df.head(2)

    cycle_df.to_csv('./'+i+'/'+'cycle_df_'+i+'.csv')

    cycle_df=pd.read_csv('./'+i+'/'+'cycle_df_'+i+'.csv', index_col=[0])


    cycle_df=cycle_df[cycle_df['cycle_lenght']==150]
    cycle_df=cycle_df.sort_values(['signal','start_cycle']) 
    cycle_df=cycle_df.reset_index(drop=True)



    grouped=cycle_df.groupby(['signal'])
    id_cycle_df=pd.DataFrame([])
    for sets , data in grouped:
        data=data.sort_values(['start_cycle']).reset_index(drop=True)
        data['cycle_id']=data.index
        id_cycle_df=id_cycle_df.append(data)


    id_cycle_df=id_cycle_df.sort_values(['signal','start_cycle']).reset_index(drop=True)
    id_cycle_df['unique_cycle_id']=id_cycle_df.index
    
    id_cycle_df.to_csv('./'+i+'/'+'id_cycle_df'+i+'.csv')
    #id_cycle_df
    occupancy=df[(df['event']==82) | (df['event']==81)].sort_values(by=['signal','parameter','datetime'])
    occupancy['code_shift']=occupancy.groupby(['signal','parameter'])['event'].shift(-1)

    occupancy['detector_on_end']=occupancy.groupby(['signal','parameter'])['datetime'].shift(-1)




    id_cycle_df=id_cycle_df.sort_values(['signal','start_cycle']).reset_index(drop=True)
    id_cycle_df['unique_cycle_id']=id_cycle_df.index
    id_cycle_df

    occupancy.to_csv('./'+i+'/'+'occupancy'+i+'.csv')

    detector=occupancy[(occupancy['code_shift']==81)  & (occupancy['event']==82)]


    detector=detector.rename(columns={'datetime':'detector_on_start','parameter':'detector_parameter'})

    grouped=detector.groupby(['signal','detector_parameter'])
    detector_df=pd.DataFrame([])
    for sets , data in grouped:
        data=data.sort_values(['detector_on_start']).reset_index(drop=True)
        data['detector_id']=data.index
        detector_df=detector_df.append(data)
    detector_df=detector_df.sort_values(['signal','detector_parameter','detector_on_start']).reset_index(drop=True)
    detector_df['unique_detector_id']=detector_df.index    


    detector_df.to_csv('./'+i+'/'+'detector_df'+i+'.csv')

    import sqlite3
    #Make the db in memory
    cycle_db = sqlite3.connect(':memory:')
    #write the tables
    detector_df.to_sql('detector', cycle_db, index=False)
    id_cycle_df.to_sql('cycle_df', cycle_db, index=False)

    qry = '''
        select  
            detector.*
            ,cycle_df.start_cycle
            ,cycle_df.end_cycle
            ,cycle_df.signal
            ,cycle_df.cycle_id
            ,cycle_df.unique_cycle_id
        from detector 
            join cycle_df  using(signal)
            WHERE
            detector_on_start between start_cycle and end_cycle
            OR
            detector_on_end between start_cycle and end_cycle

        '''


    df_query = pd.read_sql_query(qry, cycle_db)
    df_query = df_query.loc[:,~df_query.columns.duplicated()]


    df_query.to_csv('./'+i+'/'+'df_query'+i+'.csv')

    df_query['start_cycle']=pd.to_datetime(df_query['start_cycle'])
    df_query['end_cycle']=pd.to_datetime(df_query['end_cycle'])
    df_query['detector_on_start']=pd.to_datetime(df_query['detector_on_start'])
    df_query['detector_on_end']=pd.to_datetime(df_query['detector_on_end'])


    a_query=df_query[['detector_on_start','detector_on_end','start_cycle','end_cycle','cycle_id','unique_cycle_id','signal','detector_parameter','detector_id','unique_detector_id']]
    a_query['round']=np.floor((a_query['end_cycle']-a_query['detector_on_end']).dt.total_seconds()/5)
    a_query['end_chunk']=30- (np.floor((a_query['end_cycle']-a_query['detector_on_end']).dt.total_seconds()/5))
    a_query['start_chunk']=np.ceil((a_query['detector_on_start']-a_query['start_cycle']).dt.total_seconds()/5)
    # because of ceiling issue , when detector on time is the same as cycle start time it will give 0 we push that to 1 
    # manually below
    a_query['start_chunk']=np.where(a_query['start_chunk']==0,1,a_query['start_chunk'])


    ###start chunk 2 for the later cycle 
    a_query['start_chunk_2']=1





    a_query['start_chunk_seconds']= (pd.TimedeltaIndex((a_query['start_chunk']*5), unit='s') +a_query['start_cycle']-a_query['detector_on_start']).dt.total_seconds()
    a_query['end_chunk_seconds']=(a_query['detector_on_end']-a_query['start_cycle']).dt.total_seconds()-(a_query['end_chunk']-1)*5
    a_query['on_the_same_chunk']=np.where(a_query['start_chunk']==a_query['end_chunk'],(a_query['detector_on_end']-a_query['detector_on_start']).dt.total_seconds(),'na')


    a_query.to_csv('./'+i+'/'+'a_query'+i+'.csv')


    a_query['different_cyles']=np.where(a_query['detector_on_end']>a_query['end_cycle'],'yes','no')
    # detector start before cycle start
    a_query['different_cyles_2']=np.where(a_query['detector_on_start']<a_query['start_cycle'],'yes','no')


    a_query=a_query.sort_values(['signal','detector_parameter','unique_detector_id']).reset_index(drop=True)

    #a_query['different_cyles_2_shift']=a_query.groupby(['signal','detector_parameter','unique_detector_id'])['different_cyles_2'].shift(-1)

    #a_query['detector_on_start_shift']=a_query.groupby(['signal','detector_parameter','unique_detector_id'])['detector_on_start'].shift(-1)
    #a_query['same_detector_start']=np.where(a_query['detector_on_start_shift']==a_query['detector_on_start'],'yes','no')

    # select detectors that does not start and end on the same  cycle 

    a_query=a_query[(a_query['detector_on_start'] >= a_query['start_cycle']) &  (a_query['detector_on_end'] > a_query['end_cycle'])  | (a_query['detector_on_start'] < a_query['start_cycle']) &  (a_query['detector_on_end'] <= a_query['end_cycle'])  ]


    a_query.to_csv('./'+i+'/'+'a_query_1'+i+'.csv')

    a_query['below_detection']=a_query.groupby(['signal','detector_parameter','unique_detector_id'])['unique_detector_id'].shift(-1)

    a_query['above_detection']=a_query.groupby(['signal','detector_parameter','unique_detector_id'])['unique_detector_id'].shift(1)

    #for those detections that start or end on a cycle that is not under this study (between 90 and 120)
    a_query_above_below_null=a_query[a_query['above_detection'].isnull() & a_query['below_detection'].isnull()]


    a_query.to_csv('./'+i+'/'+'a_query_2'+i+'.csv')
    a_query_above_below_null.to_csv('./'+i+'/'+'a_query_above_below_null'+i+'.csv')

    a_query_above_below_null['which_cycle']='na'

    first_condition=(a_query_above_below_null['detector_on_end'] > a_query_above_below_null['end_cycle'])
    a_query_above_below_null['which_cycle']=np.where(a_query_above_below_null['detector_on_end'] > a_query_above_below_null['end_cycle'],'first','second')

    a_query_above_below_null=a_query_above_below_null[(a_query_above_below_null['detector_on_start'] < a_query_above_below_null['end_cycle']) & (a_query_above_below_null['detector_on_start'] > a_query_above_below_null['start_cycle']) | (a_query_above_below_null['detector_on_end'] < a_query_above_below_null['end_cycle']) & (a_query_above_below_null['detector_on_end'] > a_query_above_below_null['start_cycle']) ]

    a_query_above_below_null.to_csv('./'+i+'/'+'a_query_above_below_null'+i+'.csv')

    def range_creation(x):
        if (x['which_cycle']=='first'):
            out=list(np.arange(x['start_chunk'], 31, 1))
            return(out)
        else :
            out=list(np.arange(1, x['end_chunk']+1, 1))
            return(out)


    grouped=a_query_above_below_null.groupby(['signal','detector_parameter'])   
    null_cycles_detectors=pd.DataFrame([])
    for sets ,data  in grouped:
        data['l']=data.apply(lambda x: range_creation(x),axis=1)
        null_cycles_detectors=null_cycles_detectors.append(data)
    #t

    null_cycles_detectors.to_csv('./'+i+'/'+'null_different_cycles_detectors'+i+'.csv')

    null_left=null_cycles_detectors.l.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True).to_frame('b').reset_index()
    null_left_merged=null_left.join(null_cycles_detectors, how='left',on='index', lsuffix='_left', rsuffix='_right')
    null_left_merged.to_csv('./'+i+'/'+'null_different_cycles_detectors_2'+i+'.csv')

    null_left_merged['location']='NA'
    def location_set_null(x):
        if (x.which_cycle.unique()=='first'):
            first_group=x[x['which_cycle']== 'first'] 
                #out=1
            first_group['location'].iloc[0]='first'
                    #out='first'
                #if (x.iloc[1:]):
            first_group['location'].iloc[1:]='between'
            return(first_group)
                    #ut='between'
        #else:
            #out=2
        #print(first_group)
        if (x.which_cycle.unique()=='second'):
            second_group=x[x['which_cycle']== 'second']     
            second_group['location'].iloc[0:-1]='between'
            second_group['location'].iloc[-1]='last'
            #print(pd.concat([first_group,second_group]))
            return(second_group)
        #return(second_group)


    import time
    start = time.time()
    grouped=null_left_merged.groupby(['unique_detector_id'])
    last=pd.DataFrame([])
    for sets, data in grouped:
        temp=location_set_null(data)
        last=last.append(temp)
        print(sets)
    end = time.time()
    print(end - start)


    last['on_the_same_chunk']=np.where(last['start_chunk']==last['end_chunk'],(last['detector_on_end']-last['start_cycle']).dt.total_seconds(),'na')


    left_merged=last[['signal','detector_parameter','detector_on_start','detector_on_end','location','b','cycle_id','unique_cycle_id','start_cycle','end_cycle','start_chunk_seconds','end_chunk_seconds','detector_id','unique_detector_id','on_the_same_chunk']]
    left_merged['seconds']='na'
    left_merged['seconds']=np.where(left_merged['location']=='first',left_merged['start_chunk_seconds'],left_merged['seconds'])
    left_merged['seconds']=np.where(left_merged['location']=='last',left_merged['end_chunk_seconds'],left_merged['seconds'])
    left_merged['seconds']=np.where(left_merged['location']=='between',5,left_merged['seconds'])


    left_merged['seconds']=np.where(left_merged['on_the_same_chunk']!='na',left_merged['on_the_same_chunk'],left_merged['seconds'])



    left_merged['seconds']=np.where(left_merged['location']=='between',5,left_merged['seconds'])


    left_merged.to_csv('./'+i+'/'+'null_alternative_solution_different_cycle'+i+'.csv')



    a_query['below_detection']=a_query.groupby(['signal','detector_parameter','unique_detector_id'])['unique_detector_id'].shift(-1)

    a_query['above_detection']=a_query.groupby(['signal','detector_parameter','unique_detector_id'])['unique_detector_id'].shift(1)

    a_query=a_query[a_query['above_detection'].notnull()  | a_query['below_detection'].notnull()]


    a_query.to_csv('./'+i+'/'+'a_query_3'+i+'.csv')


    a_query['start_chunk']=a_query['start_chunk'].astype('int')
    a_query['end_chunk']=a_query['end_chunk'].astype('int')

    a_query[a_query['unique_detector_id']==a_query['below_detection']]

    a_query['which_cycle']='na'
    a_query['which_cycle']=np.where(a_query['unique_detector_id']==a_query['below_detection'],'first','second')


    def range_creation(x):
        if (x['which_cycle']=='first'):
            out=list(np.arange(x['start_chunk'], 31, 1))
            return(out)
        else :
            out=list(np.arange(1, x['end_chunk']+1, 1))
            return(out)


    grouped=a_query.groupby(['signal','detector_parameter'])   
    different_cycles_detectors=pd.DataFrame([])
    for sets ,data  in grouped:
        data['l']=data.apply(lambda x: range_creation(x),axis=1)
        different_cycles_detectors=different_cycles_detectors.append(data)

    different_cycles_detectors.to_csv('./'+i+'/'+'different_cycles_detectors'+i+'.csv')	
    left=different_cycles_detectors.l.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True).to_frame('b').reset_index()
    left_merged=left.join(different_cycles_detectors, how='left',on='index', lsuffix='_left', rsuffix='_right')

    left_merged.to_csv('./'+i+'/'+'different_cycles_detectors_2'+i+'.csv')

    left_merged['location']='NA'
    def location_set(x):

        first_group=x[x['which_cycle']== 'first'] 
            #out=1
        first_group['location'].iloc[0]='first'
                #out='first'
            #if (x.iloc[1:]):
        first_group['location'].iloc[1:]='between'
                #ut='between'
        #else:
            #out=2
        #print(first_group)
        second_group=x[x['which_cycle']== 'second']     
        second_group['location'].iloc[0:-1]='between'
        second_group['location'].iloc[-1]='last'
        #print(pd.concat([first_group,second_group]))
        return(pd.concat([first_group,second_group]))
        #return(second_group)


    import time
    start = time.time()
    grouped=left_merged.groupby(['unique_detector_id'])
    last=pd.DataFrame([])
    for sets, data in grouped:
        temp=location_set(data)
        last=last.append(temp)
        print(sets)
    end = time.time()
    print(end - start)

    left_merged=last[['signal','detector_parameter','detector_on_start','detector_on_end','location','b','cycle_id','unique_cycle_id','start_cycle','end_cycle','start_chunk_seconds','end_chunk_seconds','detector_id','unique_detector_id']]
    left_merged['seconds']='na'
    left_merged['seconds']=np.where(left_merged['location']=='first',left_merged['start_chunk_seconds'],left_merged['seconds'])
    left_merged['seconds']=np.where(left_merged['location']=='last',left_merged['end_chunk_seconds'],left_merged['seconds'])
    left_merged['seconds']=np.where(left_merged['location']=='between',5,left_merged['seconds'])



    left_merged.to_csv('./'+i+'/'+'alternative_solution_different_cycle'+i+'.csv')


    conditions=(df_query['detector_on_start'] >=df_query['start_cycle']) & (df_query['detector_on_start'] <= df_query['end_cycle']) & (df_query['detector_on_end'] >=df_query['start_cycle']) & (df_query['detector_on_end'] <= df_query['end_cycle'])
    df_query_same = df_query[conditions ]

    len(df_query), len(df_query_same), len(a_query)

    df_query_same.to_csv('./'+i+'/'+'df_query_same'+i+'.csv')

    a_query_same=df_query_same[['detector_on_start','detector_on_end','start_cycle','end_cycle','cycle_id','unique_cycle_id','signal','detector_parameter','detector_id','unique_detector_id']]
    a_query_same['start_cycle']=pd.to_datetime(a_query_same['start_cycle'])
    a_query_same['end_cycle']=pd.to_datetime(a_query_same['end_cycle'])
    a_query_same['detector_on_start']=pd.to_datetime(a_query_same['detector_on_start'])
    a_query_same['detector_on_end']=pd.to_datetime(a_query_same['detector_on_end'])




    a_query_same['round']=np.floor((a_query_same['end_cycle']-a_query_same['detector_on_end']).dt.total_seconds()/5)
    a_query_same['end_chunk']=30- (np.floor((a_query_same['end_cycle']-a_query_same['detector_on_end']).dt.total_seconds()/5))
    a_query_same['start_chunk']=np.ceil((a_query_same['detector_on_start']-a_query_same['start_cycle']).dt.total_seconds()/5)
    a_query_same['start_chunk_seconds']= (pd.TimedeltaIndex((a_query_same['start_chunk']*5), unit='s') +a_query_same['start_cycle']-a_query_same['detector_on_start']).dt.total_seconds()
    a_query_same['end_chunk_seconds']=(a_query_same['detector_on_end']-a_query_same['start_cycle']).dt.total_seconds()-(a_query_same['end_chunk']-1)*5
    a_query_same['on_the_same_chunk']=np.where(a_query_same['start_chunk']==a_query_same['end_chunk'],(a_query_same['detector_on_end']-a_query_same['detector_on_start']).dt.total_seconds(),'na')

    #to account for when detectors and cycle start at same time which results in chunk start zero we push that to 1
    a_query_same['start_chunk']=np.where(a_query_same['start_chunk']==0,1,a_query_same['start_chunk'])
    b_query_same=a_query_same[['detector_on_start','detector_on_end','start_cycle','end_cycle','start_chunk','start_chunk_seconds','end_chunk','end_chunk_seconds','on_the_same_chunk','cycle_id','unique_cycle_id','signal','detector_parameter','detector_id','unique_detector_id']]

    # on the same chunk
    b_query_different=b_query_same[b_query_same['on_the_same_chunk']!='na']

    #different chunks
    b_query_same=b_query_same[b_query_same['on_the_same_chunk']=='na']


    b_query_same.to_csv('./'+i+'/'+'b_query_same'+i+'.csv')
    b_query_different.to_csv('./'+i+'/'+'b_query_different'+i+'.csv')




    b_query_same['start_chunk']=b_query_same['start_chunk'].astype('int')
    b_query_same['end_chunk']=b_query_same['end_chunk'].astype('int')


    b_query_same['l']=b_query_same.apply(lambda x: list(np.arange(x['start_chunk'], x['end_chunk']+1, 1)),axis=1)
    b_query_same['index']=b_query_same.index


    left_same=b_query_same.l.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True).to_frame('b').reset_index()

    left_same['location']='NA'
    def location_set(x):
        x.location.iloc[0]='first'
        x['location'][1:-1]='between'
        x['location'].iloc[-1]='last'
        return(x)


    left_same=left_same.groupby('index').apply(location_set)


    left_merged_same=left_same.join(b_query_same, how='left',on='index', lsuffix='_left', rsuffix='_right')

    left_merged_same=left_merged_same[['signal','detector_parameter','detector_on_start','detector_on_end','location','b','cycle_id','unique_cycle_id','start_cycle','end_cycle','start_chunk_seconds','end_chunk_seconds','detector_id','unique_detector_id']]

    left_merged_same['seconds']='na'
    left_merged_same['seconds']=np.where(left_merged_same['location']=='first',left_merged_same['start_chunk_seconds'],left_merged_same['seconds'])
    left_merged_same['seconds']=np.where(left_merged_same['location']=='last',left_merged_same['end_chunk_seconds'],left_merged_same['seconds'])
    left_merged_same['seconds']=np.where(left_merged_same['location']=='between',5,left_merged_same['seconds'])


    left_merged_same.to_csv('./'+i+'/'+'alternative_solution_same_cylce'+i+'.csv')
    b_query_different=b_query_different.rename(columns={'on_the_same_chunk':'seconds', 'start_chunk':'chunk'})
    same_chunk_same_cycle=b_query_different[['signal','detector_parameter','detector_on_start','detector_on_end','cycle_id','unique_cycle_id','start_cycle','end_cycle','start_chunk_seconds','end_chunk_seconds','detector_id','unique_detector_id','seconds','chunk']]

    same_chunk_same_cycle.to_csv('./'+i+'/'+'same_chunk_same_cycle'+i+'.csv')




    same_chunk_same_cycle=pd.read_csv('./'+i+'/'+'same_chunk_same_cycle'+i+'.csv', index_col=[0])
    alternative_solution_same_cylce=pd.read_csv('./'+i+'/'+'alternative_solution_same_cylce'+i+'.csv', index_col=[0])
    alternative_solution_different_cycle=pd.read_csv('./'+i+'/'+'alternative_solution_different_cycle'+i+'.csv', index_col=[0])
    null_alternative_solution_different_cycle=pd.read_csv('./'+i+'/'+'null_alternative_solution_different_cycle'+i+'.csv', index_col=[0])




    null_alternative_solution_different_cycle=null_alternative_solution_different_cycle.rename(columns={'b':'chunk'})
    alternative_solution_different_cycle=alternative_solution_different_cycle.rename(columns={'b':'chunk'})
    alternative_solution_same_cylce=alternative_solution_same_cylce.rename(columns={'b':'chunk'})

    null_alternative_solution_different_cycle=null_alternative_solution_different_cycle[['signal','detector_parameter','cycle_id','unique_cycle_id','start_cycle','end_cycle','unique_detector_id','chunk','seconds']]
    alternative_solution_different_cycle=alternative_solution_different_cycle[['signal','detector_parameter','cycle_id','unique_cycle_id','start_cycle','end_cycle','unique_detector_id','chunk','seconds']]
    alternative_solution_same_cylce=alternative_solution_same_cylce[['signal','detector_parameter','cycle_id','unique_cycle_id','start_cycle','end_cycle','unique_detector_id','chunk','seconds']]
    same_chunk_same_cycle=same_chunk_same_cycle[['signal','detector_parameter','cycle_id','unique_cycle_id','start_cycle','end_cycle','unique_detector_id','chunk','seconds']]

    combined=pd.concat([null_alternative_solution_different_cycle,alternative_solution_different_cycle,alternative_solution_same_cylce,same_chunk_same_cycle])

    t=combined.groupby(['signal','detector_parameter','cycle_id','unique_cycle_id','start_cycle','end_cycle','chunk'])['seconds'].sum().reset_index()

    Detectors=pd.read_csv('../../meta/Detectors.csv')
    approaches=pd.read_csv('../../meta/approaches.csv')
    Detectors=Detectors.merge(approaches, how='left', on='ApproachID')
    DetectionTypeDetector=pd.read_csv('../../meta/DetectionTypeDetector.csv')
    DetectionTypes=pd.read_csv('../../meta/DetectionTypes.csv')
    DetectionTypeDetector=DetectionTypeDetector.merge(DetectionTypes, how='left', on=['DetectionTypeID'])
    Detectors=Detectors.merge(DetectionTypeDetector, how='left',on='ID')


    helper=Detectors[['DetChannel','LaneNumber','SignalID','ProtectedPhaseNumber','Description_x','Description_y', 'DistanceFromStopBar']]
    f=t.merge(helper, how='left', left_on=['signal','detector_parameter'], right_on=['SignalID','DetChannel']).drop(['SignalID'],axis=1)


    f.to_csv('./'+i+'/'+'occupancy_of_chunk_150_lenght'+i+'.csv')




C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

214
217
272
3461
3903
9646
14195
14225
14288
14861
14909
14912
15001
24637
29339
32074
32373
32404
33097
37589
42432
43331
44590
44599
44751
50333
51238
60100
65838
65904
66056
66876
66951
66958
67129
67394
72108
79419
88672
90334
90426
92856
98812
98940
99985
100389
101732
102149
103435
104489
104545
105328
105463
129158
129279
130749
138037
138932
140233
140687
142250
142805
144068
144416
147119
147233
147950
148053
201609
201863
202944
203321
223954
224292
226099
226526
229991
230946
234439
235278
238054
240254
240641
242475
242947
262165
263928
265986
266467
269733
270919
275733
278097
278725
282582
282603
286340
286567
292089
292121
297081
298374
298422
298425
298454
298687
299144
308653
314414
314416
314460
317274
321354
324891
326397
327209
327257
327705
327780
339233
339297
345255
360173
360295
360301
360368
360778
361386
362170
362174
362227
381238
382302
382792
382935
383468
383483
383485
383498
383501
383503
383505
383542
386273
386467
386489
386495
394593
397706
398818
3988

C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

195
196
198
200
202
203
204
205
206
207
209
211
212
213
218
219
220
221
222
223
224
225
226
227
229
231
232
234
235
236
237
238
240
241
243
248
249
251
253
254
255
256
258
259
260
261
262
264
267
268
269
270
271
3280
3357
3383
3389
3438
3485
3527
3532
3556
3581
3586
3602
3626
3634
3645
3655
3677
3687
3726
3759
3768
3788
3852
3866
3882
8696
8750
8765
8901
8941
8970
8979
9027
9040
9072
9118
9126
9164
9197
9313
9337
9380
9433
9519
9530
9546
9604
12906
12943
12952
12979
13036
13043
13050
13128
13147
13319
14196
14198
14199
14201
14202
14203
14205
14206
14207
14209
14211
14212
14213
14214
14215
14217
14218
14219
14221
14226
14228
14229
14230
14234
14235
14237
14238
14239
14240
14241
14245
14246
14247
14249
14250
14252
14253
14254
14256
14257
14260
14261
14264
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14277
14278
14279
14280
14282
14283
14285
14287
14864
14869
14871
14874
14877
14880
14881
14884
14888
14891
14892
14894
14897
14899
14900
14901
14903
14904
14905
14907
14913
1

140430
140434
140437
140443
140447
140457
140467
140471
140483
140492
140505
140514
140519
140530
140536
140546
140549
140559
140564
140569
140571
140573
140584
140592
140598
140605
140609
140616
140623
140627
140635
140638
140643
140650
140653
140663
140668
140671
140675
140681
142255
142259
142266
142274
142282
142286
142293
142301
142311
142320
142328
142335
142344
142353
142359
142368
142376
142384
142396
142409
142424
142433
142442
142449
142457
142467
142475
142485
142496
142502
142513
142520
142526
142540
142554
142559
142565
142572
142581
142587
142596
142608
142614
142627
142638
142644
142649
142657
142667
142675
142677
142687
142695
142697
142704
142709
142715
142725
142730
142735
142745
142748
142756
142765
142773
142779
142786
142792
142802
144072
144081
144086
144088
144091
144092
144097
144102
144106
144114
144118
144122
144125
144130
144132
144140
144144
144153
144156
144158
144161
144170
144173
144177
144184
144187
144193
144203
144210
144220
144223
144229
144237
144242

...

605977
605983
605987
605988
605989
605991
605994
605997
606001
606003
606004
606005
606006
606879
606880
606883
606884
606886
606887
606890
606894
606896
606897
606900
606902
606905
606908
606910
606912
606914
606915
606919
606920
606923
606925
606926
606932
606933
606935
606937
606945
606947
606949
606950
606953
606956
606958
606960
606961
606963
606967
606970
606971
606974
606977
606979
606982
606984
606986
606989
606994
606998
607001
607005
607008
607012
607015
607016
607017
636487
636568
636630
636759
636913
636983
637047
637189
637261
637343
637497
637566
637626
637700
637843
637916
637990
638063
638127
638274
638348
638498
639058
639195
639254
639313
639454
639521
639667
639738
639863
639926
639998
640356
640497
640629
640690
640990
641127
641199
653349
653356
653367
653387
653390
653402
653406
653410
653415
653419
653423
653426
653431
653435
653440
653449
653460
653466
653483
653490
653497
653499
653528
653532
653536
653540
653545
653550
653557
653561
653564
653565
653576
653578

774414
774419
774426
774427
774429
774433
774438
774442
774445
774448
774453
774457
774464
774465
774471
774474
774476
774477
774479
774483
774486
774489
774490
774492
774497
774499
774500
774502
774505
774509
774510
774514
774516
774520
774522
774524
774527
774528
774534
784586
784640
784774
784828
785028
785123
785221
785275
785539
785578
785666
785768
785959
786010
786050
786154
786348
786491
787080
787128
787263
803720
803764
803810
804038
804517
804612
804756
804951
805033
805131
812224
812228
812233
812236
812239
812244
812250
812253
812260
812270
812275
812278
812282
812285
812291
812299
812308
812311
812317
812320
812324
812329
812334
812338
812342
812345
812355
812362
812367
812371
812378
812388
812393
812401
812405
812408
812415
812417
812419
816659
817268
823969
823987
824052
824357
824559
824888
832216
832300
833041
833350
833426
837365
838399
838402
838409
838412
838416
838423
838433
838435
838437
838438
838445
838448
838452
838455
838460
838464
838466
838467
838474
838477

992294
992297
992302
992312
992317
992324
992337
992345
992350
992356
992362
992364
992368
992373
992378
992386
992392
992400
992405
992414
992419
992428
992437
992440
992445
992449
992454
992456
992458
992462
992465
992472
992477
992483
997088
1014924
1015067
1015221
1015480
1018752
1018753
1018755
1018756
1018759
1018763
1018765
1018766
1018768
1018769
1018770
1018772
1018773
1018774
1018777
1018779
1018781
1018782
1018783
1018798
1018799
1018800
1018801
1018802
1018803
1018807
1018808
1018809
1018811
1018813
1018814
1018815
1018818
1018830
1018832
1018833
1018834
1018835
1018839
1021300
1021307
1021311
1021322
1021332
1021336
1021339
1021348
1021405
1021416
1021420
1021457
1021470
1021479
1021504
1021551
1021607
1021618
1021661
1021670
1021712
1021764
1021796
1021845
1021855
1021932
1021953
1021968
1021979
1022060
1025475
1025484
1025522
1025538
1025575
1025616
1025707
1025719
1025750
1025761
1025794
1025806
1025816
1025899
1025916
1025924
1025934
1025947
1025961
1026002
1026010
102

1145273
1145332
1145354
1145437
1145578
1149767
1149779
1149788
1149799
1149812
1149823
1149833
1149851
1149862
1149872
1149880
1149891
1149902
1149912
1149924
1149935
1149943
1149956
1149970
1149981
1149994
1150006
1150017
1150027
1150040
1150051
1150059
1150072
1150081
1150091
1150150
1150159
1150168
1150179
1150189
1150202
1150213
1150225
1150235
1150245
1150253
1150263
1150273
1150281
1150293
1150301
1150344
1150358
1150408
1150420
1150429
1150439
1150449
1150458
1150471
1150480
1150494
1150506
1150518
1153900
1153911
1153922
1153931
1153941
1153951
1153962
1153972
1153984
1153994
1154005
1154015
1154027
1154038
1154048
1154057
1154067
1154077
1154086
1154099
1154107
1154116
1154126
1154136
1154146
1154159
1154170
1154184
1154193
1154204
1154249
1154259
1154269
1154279
1154288
1154299
1154308
1154316
1154324
1154332
1154339
1154347
1154356
1154364
1154370
1154372
1154402
1154409
1154451
1154457
1154466
1154475
1154483
1154485
1154491
1154495
1154505
1154512
1154519
1156830
1156835


1284156
1284162
1284167
1284173
1284176
1284179
1284180
1284183
1284184
1284188
1284190
1284194
1284195
1284198
1286088
1286093
1286107
1286113
1286120
1286124
1286128
1286138
1286142
1286150
1286161
1286167
1286170
1286174
1286180
1286189
1286200
1286208
1286215
1286223
1286228
1286238
1286246
1286268
1286277
1286284
1286290
1286300
1286310
1286319
1286331
1286340
1286348
1286364
1286368
1286375
1286384
1286390
1286401
1286414
1286426
1286436
1286445
1286449
1286456
1286466
1286470
1286479
1286489
1286490
1286495
1286500
1286506
1286515
1286520
1286528
1286543
1286549
1286553
1286560
1286577
1286583
1286590
1286592
1295526
1295721
1295770
1295849
1295928
1295963
1296002
1296161
1296200
1296244
1296317
1296410
1296582
1308892
1308935
1309061
1309405
1309523
1309569
1309784
1309833
1309877
1310123
1310402
1310611
1310754
1310883
1310967
1311097
1311302
1311413
1311676
1322461
1322525
1322552
1322660
1323065
1323463
1323796
1324040
1324137
1324166
1324385
1324524
1327771
1327775
1327788
